Consigne : Visualisation des recettes par réduction de dimension pour déterminer les recettes qui se rapprochent de celles que l'on a déjà faites.
Réaliser votre étude en écrivant le code python permettant d’analyser ces données et répondre à vos questions :
○ Sélectionner les variables d'intérêt
○ Manipuler la donnees afin d’extraire les statistiques / insights intéressants
○ Visualiser ces résultats dans des visualisation appropriées et dynamiques (vous pouvez par exemple utiliser Plotly)

1. Sélectionner les variables d’intérêt
L’idée est de représenter chaque recette comme un vecteur numérique qui encode ses caractéristiques.
 Variables possibles :
Liste des ingrédients (présence/absence → encodage binaire type Bag of Ingredients).
Temps de préparation / cuisson (normalisés).
Catégorie de recette (entrée, plat, dessert → encodage one-hot).
Nombre d’ingrédients.
:point_right: Résultat attendu : un DataFrame numérique (X) avec une ligne par recette.

In [2]:
import pandas as pd
from IPython.display import display  # pour afficher joliment dans un notebook

# (Optionnel) un peu de confort d'affichage
pd.set_option("display.max_columns", None)
pd.set_option("display.width", 0)

# Charger le CSV
df = pd.read_csv("Data/RAW_recipes.csv")

# Dimensions du dataset
print("Shape (lignes, colonnes) :", df.shape)
print("-------------------------------------------------------------------------------------")

# Colonnes numériques pour le dégradé
num_cols = df.select_dtypes(include="number").columns

# Aperçu stylé des 10 premières lignes
styled = (
    df.head(10)
      .style
      .background_gradient(cmap="Blues", subset=num_cols)  # dégradé sur colonnes numériques
      .hide(axis="index")                                   # masque l'index pour un rendu plus clean
)

display(styled)


Shape (lignes, colonnes) : (231637, 12)
-------------------------------------------------------------------------------------


name,id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients
arriba baked winter squash mexican style,137739,55,47892,2005-09-16,"['60-minutes-or-less', 'time-to-make', 'course', 'main-ingredient', 'cuisine', 'preparation', 'occasion', 'north-american', 'side-dishes', 'vegetables', 'mexican', 'easy', 'fall', 'holiday-event', 'vegetarian', 'winter', 'dietary', 'christmas', 'seasonal', 'squash']","[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]",11,"['make a choice and proceed with recipe', 'depending on size of squash , cut into half or fourths', 'remove seeds', 'for spicy squash , drizzle olive oil or melted butter over each cut squash piece', 'season with mexican seasoning mix ii', 'for sweet squash , drizzle melted honey , butter , grated piloncillo over each cut squash piece', 'season with sweet mexican spice mix', 'bake at 350 degrees , again depending on size , for 40 minutes up to an hour , until a fork can easily pierce the skin', 'be careful not to burn the squash especially if you opt to use sugar or butter', 'if you feel more comfortable , cover the squash with aluminum foil the first half hour , give or take , of baking', 'if desired , season with salt']","autumn is my favorite time of year to cook! this recipe can be prepared either spicy or sweet, your choice! two of my posted mexican-inspired seasoning mix recipes are offered as suggestions.","['winter squash', 'mexican seasoning', 'mixed spice', 'honey', 'butter', 'olive oil', 'salt']",7
a bit different breakfast pizza,31490,30,26278,2002-06-17,"['30-minutes-or-less', 'time-to-make', 'course', 'main-ingredient', 'cuisine', 'preparation', 'occasion', 'north-american', 'breakfast', 'main-dish', 'pork', 'american', 'oven', 'easy', 'kid-friendly', 'pizza', 'dietary', 'northeastern-united-states', 'meat', 'equipment']","[173.4, 18.0, 0.0, 17.0, 22.0, 35.0, 1.0]",9,"['preheat oven to 425 degrees f', 'press dough into the bottom and sides of a 12 inch pizza pan', 'bake for 5 minutes until set but not browned', 'cut sausage into small pieces', 'whisk eggs and milk in a bowl until frothy', 'spoon sausage over baked crust and sprinkle with cheese', 'pour egg mixture slowly over sausage and cheese', 's& p to taste', 'bake 15-20 minutes or until eggs are set and crust is brown']",this recipe calls for the crust to be prebaked a bit before adding ingredients. feel free to change sausage to ham or bacon. this warms well in the microwave for those late risers.,"['prepared pizza crust', 'sausage patty', 'eggs', 'milk', 'salt and pepper', 'cheese']",6
all in the kitchen chili,112140,130,196586,2005-02-25,"['time-to-make', 'course', 'preparation', 'main-dish', 'chili', 'crock-pot-slow-cooker', 'dietary', 'equipment', '4-hours-or-less']","[269.8, 22.0, 32.0, 48.0, 39.0, 27.0, 5.0]",6,"['brown ground beef in large pot', 'add chopped onions to ground beef when almost brown and sautee until wilted', 'add all other ingredients', 'add kidney beans if you like beans in your chili', 'cook in slow cooker on high for 2-3 hours or 6-8 hours on low', 'serve with cold clean lettuce and shredded cheese']",this modified version of 'mom's' chili was a hit at our 2004 christmas party. we made an extra large pot to have some left to freeze but it never made it to the freezer. it was a favorite by all. perfect for any cold and rainy day. you won't find this one in a cookbook. it is truly an original.,"['ground beef', 'yellow onions', 'diced tomatoes', 'tomato paste', 'tomato soup', 'rotel tomatoes', 'kidney beans', 'water', 'chili powder', 'ground cumin', 'salt', 'lettuce', 'cheddar cheese']",13
alouette potatoes,59389,45,68585,2003-04-14,"['60-minutes-or-less', 'time-to-make', 'course', 'main-ingredient', 'preparation', 'occasion', 'side-dishes', 'eggs-dairy', 'potatoes', 'vegetables', 'oven', 'easy', 'dinner-party', 'holiday-event', 'easter', 'cheese', 'stove-top', 'dietary', 'christmas', 'new-years', 'thanksgiving', 'independence-day', 'st-patricks-day', 'val

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.feature_extraction.text import CountVectorizer
from scipy.sparse import hstack

# Charger le dataset
df = pd.read_csv("Data/RAW_recipes.csv")

# Variables d'intérêt
X = df[["temps_preparation", "temps_cuisson", "categorie", "ingredients"]]

# Encodage de la catégorie (OneHot)
encoder = OneHotEncoder()
cat_matrix = encoder.fit_transform(X[["categorie"]]).toarray()

# Normalisation du temps
scaler = StandardScaler()
time_matrix = scaler.fit_transform(X[["temps_preparation", "temps_cuisson"]])

# Encodage simple des ingrédients (CountVectorizer)
vectorizer = CountVectorizer()
ingredients_matrix = vectorizer.fit_transform(X["ingredients"].astype(str))  # cast en str au cas où

# Concaténation finale
final_matrix = hstack([ingredients_matrix, time_matrix, cat_matrix])


2. Manipuler les données
Nettoyage : gestion des valeurs manquantes (remplissage, suppression).
Normalisation / standardisation (souvent nécessaire pour PCA, t-SNE, UMAP).
Encodage des variables catégorielles (OneHotEncoder, CountVectorizer).

3. Réduction de dimension
Objectif : passer de haute dimension (ingrédients + temps + catégories) à 2D/3D pour visualiser.
Options :
PCA (rapide, linéaire)
t-SNE (non linéaire, très visuel pour clusters)
UMAP (rapide + préserve les structures globales et locales)
Exemple PCA + t-SNE :

In [ ]:
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

# PCA pour réduire à 50 dimensions avant t-SNE
pca = PCA(n_components=50)
X_pca = pca.fit_transform(final_matrix.toarray())

# t-SNE en 2D
tsne = TSNE(n_components=2, perplexity=30, random_state=42)
X_tsne = tsne.fit_transform(X_pca)

df_vis = pd.DataFrame(X_tsne, columns=["x", "y"])
df_vis["categorie"] = df["categorie"]

4. Visualisation interactive

Avec Plotly, tu peux générer des graphiques dynamiques (zoom, hover).

Exemple : scatter plot interactif coloré par catégorie :


In [ ]:
import plotly.express as px

fig = px.scatter(
    df_vis, x="x", y="y", color="categorie",
    hover_data=["categorie"],
    title="Visualisation des recettes (t-SNE)"
)
fig.show()

5. Insights possibles

Les clusters montrent des familles de recettes (ex. desserts sucrés regroupés).

Identifier les recettes atypiques (isolées).

Comparer la proximité entre types de plats → ex. plats végétariens proches des entrées légères.

Faire des focus interactifs (filtres par temps de préparation ou nb d’ingrédients).

6. Aller plus loin (optionnel)

Ajouter un clustering K-means sur l’espace réduit pour nommer les clusters.

Construire une petite app Streamlit ou Dash pour naviguer dans les clusters.

Comparer PCA vs t-SNE vs UMAP pour voir la différence des représentations.